In [3]:
import pandas as pd

In [4]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [7]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [8]:
dataset=dataset.drop("User ID",axis=1)

In [9]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [10]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [11]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [12]:
indep.shape

(400, 3)

In [13]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [14]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [15]:
from sklearn.naive_bayes import GaussianNB

In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
} 

grid = GridSearchCV(GaussianNB(), param_grid, scoring='f1', cv=5, verbose=2, n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06]},
             scoring='f1', verbose=2)

In [17]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [18]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'var_smoothing': 1e-09}: 0.9022944298888884


In [19]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[80  5]
 [ 8 41]]


In [20]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.91      0.94      0.92        85
           1       0.89      0.84      0.86        49

    accuracy                           0.90       134
   macro avg       0.90      0.89      0.89       134
weighted avg       0.90      0.90      0.90       134



In [21]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.9625450180072028)

In [22]:
table=pd.DataFrame.from_dict(re)

In [23]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006956,0.001171,0.007761,0.000392,1.000000e-09,{'var_smoothing': 1e-09},0.709677,0.810811,0.750000,0.947368,0.944444,0.832460,0.098070,1
1,0.005547,0.001459,0.005684,0.001027,1.000000e-08,{'var_smoothing': 1e-08},0.709677,0.777778,0.685714,0.947368,0.944444,0.812996,0.112650,2
2,0.004371,0.000729,0.005942,0.000816,1.000000e-07,{'var_smoothing': 1e-07},0.645161,0.620690,0.384615,0.842105,0.750000,0.648514,0.153756,3
3,0.003807,0.000586,0.006091,0.000649,1.000000e-06,{'var_smoothing': 1e-06},0.620690,0.538462,0.384615,0.666667,0.571429,0.556372,0.096280,4


In [26]:
age_input=float(input("Age:"))
salary_input=float(input("EstimatedSalary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 31
EstimatedSalary: 160000
Sex Male 0 or 1: 1


In [27]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


C:\Users\Priyan\anaconda3\envs\aiml\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
